In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [25]:
s = ['A191RL']

df = nipa_df(retrieve_table('T10502')['Data'], s)

df.to_csv(data_dir / 'gdp.csv', index_label='date')

txt = f'{df.index[-1].year} Q{df.index[-1].quarter}: {df["A191RL"].iloc[-1]}\%'

write_txt(data_dir / 'gdp.txt', txt)

In [4]:
s = ['DPCERY', 'A006RY', 'A822RY', 'A019RY']

(nipa_df(retrieve_table('T10502')['Data'], s)
 .to_csv(data_dir / 'comp.csv', index_label='date'))

In [5]:
s = ['A939RC', 'A939RX']

df = nipa_df(retrieve_table('T70100')['Data'], s)
df['value'] = (df['A939RX'] / df['A939RX'].iloc[-1])  * df['A939RC'].iloc[-1]
df[['value']].to_csv(data_dir / 'gdppc.csv', index_label='date')

In [4]:
s = ['A261RX', 'W256RX']
rgdi = nipa_df(retrieve_table('T11706')['Data'], s).dropna()

s = ['A261RC', 'A4002C', 'W056RC', 'A107RC', 'W271RC', 'A262RC']
df = nipa_df(retrieve_table('T11000')['Data'], s).dropna()

# Calculate indirect taxes net of transfers
df['indirect'] = df['W056RC'] - df['A107RC']
df = df.drop(['A107RC', 'W056RC'], axis=1)

# Calculate GDI deflator from real GDI series
deflator = rgdi['A261RX'] / df['A261RC']
deflator = deflator / deflator.iloc[-1]
df = df.multiply(deflator, axis=0)

# Calculate contributions to growth
dft = df.diff()
dft = dft.div(dft['A261RC'], axis=0)
contr = dft.multiply((((df['A261RC'].pct_change() + 1) ** 4) - 1) * 100, axis=0)
contr.round(2).to_csv(data_dir / 'gdi.csv', index_label='date')

### Sectoral Accounts

In [83]:
s = ['W162RC', 'W994RC', 'AD01RC']
df = (nipa_df(retrieve_table('T50100')['Data'], s).div(
      nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC'], axis=0)) * 100

data = pd.DataFrame()
data['PRIV'] = df['W994RC']
data['GOV'] = df['AD01RC']
data['ROW'] = -df['W162RC']
data = data.dropna()
date = f'{data.index[-1].year} Q{data.index[-1].quarter}'

In [20]:
data.dropna().to_csv(data_dir / 'sectbal.csv', index_label='date')

In [101]:
sectbaltxt = f"In {date}, the US private sector was a net lender (running a surplus) of the equivalent of {data['PRIV'].iloc[-1]:.1f} percent of GDP, compared to {data.loc['2015-01-01', 'PRIV']:.1f} percent in 2015 Q1. The rest of the world was a net lender to the US, to the equivalent of {data['ROW'].iloc[-1]:.1f} percent of GDP in {date} compared to {data.loc['2015-01-01', 'ROW']:.1f} percent in 2015 Q1. Balancing these transactions, the government (federal, state, and local combined) was a net borrower (running a deficit) of the equivalent of {data['GOV'].iloc[-1]:.1f} percent of GDP, compared to {data.loc['2015-01-01', 'GOV']:.1f} percent in 2015. "
print(sectbaltxt)

In 2019 Q1, the US private sector was a net lender (running a surplus) of the equivalent of 4.3 percent of GDP, compared to 2.3 percent in 2015 Q1. The rest of the world was a net lender to the US, to the equivalent of 2.6 percent of GDP in 2019 Q1 compared to 2.4 percent in 2015 Q1. Balancing these transactions, the government (federal, state, and local combined) was a net borrower (running a deficit) of the equivalent of -6.9 percent of GDP, compared to -4.7 percent in 2015. 


In [102]:
# Print latest date to file
f = text_dir.joinpath('sectbal.txt')
with f.open('w') as wf:
    wf.write(sectbaltxt)

### Wealth to GDP

In [103]:
import pandas_datareader.data as web
import datetime 

In [104]:
s = ['HNOREMV', 'BOGZ1FL892090005Q', 'BOGZ1LM883164105Q']
start = datetime.datetime(1989,1,1)

In [105]:
df = web.DataReader(s, 'fred', start)
df['HNOREMV'] = df['HNOREMV'] * 1000.0
df = df.rename({'BOGZ1LM883164105Q': 'CORP',
                'BOGZ1FL892090005Q': 'TOTAL'}, axis=1)
df['OTHER'] = (df['TOTAL'] - (df['HNOREMV'] + df['CORP']))

In [106]:
data = (df.div(nipa_df(retrieve_table('T10105')['Data'], ['A191RC']
               )['A191RC'], axis=0)).dropna()
data.to_csv(data_dir / 'wealthgdp.csv', index_label='date')

In [107]:
d89 = data.iloc[0].round(2)
dlt = data.iloc[-1].round(2)
date = f'{data.index[-1].year} Q{data.index[-1].quarter}'

In [110]:
wealthtxt = f"The ratio of US total wealth, excluding public lands, to GDP increased to {dlt['TOTAL']} in {date} from {d89['TOTAL']} in 1989 Q1. The same ratio for the market value of corporate equities (see\cbox{{magenta!50!violet}}) increased to a {dlt['CORP']} multiple of GDP in {date} from {d89['CORP']} in 1989 Q1. The market value of residential real estate (see\cbox{{green!80!blue}}) increased to {dlt['HNOREMV']} times GDP from {d89['HNOREMV']} in 1989. The other category (see\cbox{{cyan!35!white}}), which includes tangible assets other than residential real estate less US financial obligations to the rest of the world, decreased to {dlt['OTHER']} from {d89['OTHER']} in 1989."

In [111]:
# Print latest date to file
f = text_dir.joinpath('wealthgdp.txt')
with f.open('w') as wf:
    wf.write(wealthtxt)